In [1]:
import numpy as np
import pandas as pd 
import re 
import sys 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from scipy.spatial import distance
from pyspark.sql import Row
import matplotlib.pyplot as plt

In [2]:
conf = SparkConf()
sc =SparkContext(conf=conf) 

In [3]:
iterations = 40 
beta = 0.8

In [112]:
def page_rank(dataset_path, iterations, beta):
    
    da1 = np.loadtxt(dataset_path)
    dat1= sc.parallelize(da1)
    # collect is used only to find the max value in this case number of nodes 
    data = dat1.collect()
    max_value =0
    for row in data:
        c = row
        d = int(c[0])
        
        if (d>max_value):
            max_value =d 
            
    rank = 1/max_value
    
    tup = dat1.map(lambda y : ( int(y[0]) , int(y[1]))).distinct()
    #print(tup.take(1))
    tup1 = tup.groupByKey().sortByKey()
    #print(tup1.take(1))
    tup1 = tup1.map(lambda y : (y[0], list(y[1])))
    #print(tup1.take(5))
    tup11 = tup1.map(lambda y : (y[0], len(y[1]),y[1]))
    #print(tup11.take(1))
    
    #tup2 = dat1.map(lambda y : (int(y[1]), int(y[0]))).distinct()
    #print(tup2.take(1))
    #tup2= tup2.groupByKey()
    #print(tup2.take(1))
    #tup2 = tup2.map(lambda y : (y[0], list(y[1])))
    #print(tup2.take(1))
    #tup4 = tup11.join(tup2)
    
    tup4 = tup11.map(lambda y : (y, rank))
    #print(tup4.take(1))
    tup5 = tup4.map(lambda y :(y[0][0] , y[0][1] , y[0][2] , y[1]))
    #print(tup5.take(1))
    
    for i in range(0,iterations):
        print("iteration:" , i)
        
    #print(tup4.take(1))
        
       # temp_rdd =  tup5.map(lambda y : (y[0], y[1]*rank)).sortByKey().collect()
        #print(tup5.take(1))
        tup8 = tup5.map(lambda w: [(x,w[3]/w[1]) for x in w[2]])
        #print(tup89.take(1))
        tup9 = tup8.flatMap(lambda x:x)
        tup10 = tup9.groupByKey().mapValues(list)
        tup11 = tup10.map(lambda x: (x[0],sum(x[1])))
        tup12 = dict(tup11.sortByKey().collect())
        #print(tup89)
        #print(tup89.take(1))
        
        #tup889 = tup89.sortBy(lambda x: x[0]).collect()
        #print(tup89)
        #print(temp_rdd)
        tup5 = tup5.map(lambda y : (y[0],y[1],y[2],((1-beta)/max_value) + (beta*(tup12[y[0]] if (y[0] in tup12.keys()) else 0))))
        tup13 = tup5.map(lambda y: (y[3],y[0])).sortByKey(False).collect()
        print("top 5:")
        top = tup13[:5]
        print(top)
        print("least 5:")
        bottom = tup13[-5:]
        print(bottom)
        
        
        #print(tup5.take(1))
        

In [113]:
sample1 = page_rank("graph-small.txt",iterations , beta)

iteration: 0
top 5:
[(0.03558508158508159, 14), (0.03521478521478523, 53), (0.03472646569117157, 40), (0.03270352980352981, 66), (0.02996744693215281, 27)]
least 5:
[(0.004714285714285714, 89), (0.004399999999999999, 81), (0.004371428571428571, 67), (0.004342857142857143, 80), (0.004193939393939394, 23)]
iteration: 1
top 5:
[(0.03542563135651835, 53), (0.03539158367861755, 14), (0.03188035845509592, 40), (0.030828998325441573, 1), (0.02878530001010026, 27)]
least 5:
[(0.004076923396776877, 94), (0.003835806764042058, 59), (0.003828248048248048, 81), (0.0037009985569985563, 37), (0.0035550332769748645, 85)]
iteration: 2
top 5:
[(0.035684743348858065, 53), (0.033842192679993055, 14), (0.03369689889179288, 40), (0.030176247201275257, 1), (0.029780301125783403, 27)]
least 5:
[(0.003943276824540802, 89), (0.0038302151369957226, 37), (0.0037460861960445712, 81), (0.003687053068049854, 59), (0.0034481980696515623, 85)]
iteration: 3
top 5:
[(0.03580413635216582, 53), (0.034301497741809336, 14)

top 5:
[(0.0357312022326716, 53), (0.03417090697259137, 14), (0.03363008718974389, 40), (0.03000597947978861, 1), (0.029720144201405382, 27)]
least 5:
[(0.003922466019802269, 89), (0.003808204291611451, 37), (0.0036953517493609916, 81), (0.0036698606601272845, 59), (0.0034096940774028216, 85)]
iteration: 28
top 5:
[(0.035731202232671594, 53), (0.034170906972591376, 14), (0.03363008718974389, 40), (0.030005979479788607, 1), (0.02972014420140538, 27)]
least 5:
[(0.003922466019802269, 89), (0.0038082042916114515, 37), (0.0036953517493609916, 81), (0.0036698606601272845, 59), (0.003409694077402821, 85)]
iteration: 29
top 5:
[(0.035731202232671594, 53), (0.034170906972591376, 14), (0.03363008718974389, 40), (0.030005979479788607, 1), (0.02972014420140538, 27)]
least 5:
[(0.003922466019802269, 89), (0.0038082042916114515, 37), (0.003695351749360991, 81), (0.0036698606601272845, 59), (0.003409694077402821, 85)]
iteration: 30
top 5:
[(0.035731202232671594, 53), (0.034170906972591376, 14), (0.0

In [115]:
sample2 = page_rank("graph-full.txt", iterations, beta)

iteration: 0
top 5:
[(0.0018759018759018757, 263), (0.0018663492063492068, 502), (0.001843125763125763, 126), (0.0018190764790764794, 285), (0.0018094549894549898, 146)]
least 5:
[(0.00042153846153846153, 408), (0.00037272727272727273, 62), (0.00037272727272727273, 424), (0.0003714285714285714, 93), (0.00032987012987012985, 558)]
iteration: 1
top 5:
[(0.00208871522584906, 263), (0.0019612553136778245, 965), (0.0018784860606765369, 537), (0.0018250404103778127, 243), (0.001812024186747996, 285)]
least 5:
[(0.0003929167721167721, 408), (0.0003743160173160173, 62), (0.0003505007114098023, 424), (0.00034146361574932996, 93), (0.00032792884460416927, 558)]
iteration: 2
top 5:
[(0.002023806689320866, 263), (0.001930221482836505, 537), (0.0019265979884266352, 965), (0.0018556711996323286, 243), (0.001841159061219935, 285)]
least 5:
[(0.0003877876917121473, 742), (0.0003602146579730995, 62), (0.00035365599001287314, 424), (0.00035036546727648763, 93), (0.0003282667952616297, 558)]
iteration: 3

top 5:
[(0.002020291181518222, 263), (0.0019433415714531525, 537), (0.0019254478071662731, 965), (0.0018526340162417364, 243), (0.0018273721700645127, 285)]
least 5:
[(0.00038779848719292117, 408), (0.00035481538649301475, 424), (0.00035314810510596025, 62), (0.00035135689375165904, 93), (0.0003286018525215321, 558)]
iteration: 26
top 5:
[(0.0020202911815182163, 263), (0.0019433415714531469, 537), (0.0019254478071662638, 965), (0.0018526340162417334, 243), (0.0018273721700645122, 285)]
least 5:
[(0.0003877984871929175, 408), (0.0003548153864930153, 424), (0.0003531481051059621, 62), (0.0003513568937516578, 93), (0.00032860185252152937, 558)]
iteration: 27
top 5:
[(0.002020291181518219, 263), (0.0019433415714531508, 537), (0.0019254478071662627, 965), (0.0018526340162417312, 243), (0.0018273721700645144, 285)]
least 5:
[(0.0003877984871929174, 408), (0.0003548153864930143, 424), (0.00035314810510596285, 62), (0.0003513568937516577, 93), (0.0003286018525215298, 558)]
iteration: 28
top 5: